<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import numpy as np
import shutil
try:
    if os.path.exists('Tutorial_IDP_SAPs'):
        print("Found existing directory, attempting to remove...")
        shutil.rmtree('Tutorial_IDP_SAPs', ignore_errors=True)
        print("Directory removed successfully")
    else:
        print("Directory doesn't exist")
except Exception as e:
    print(f"Error while trying to remove directory: {e}")

Directory doesn't exist


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install mdtraj
!pip install ipywidgets
!pip install MDAnalysis
import MDAnalysis
import ipywidgets as widgets
from IPython.display import display
import mdtraj as md
import pickle

Cloning into 'Tutorial_IDP_SAPs'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 160 (delta 53), reused 99 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (160/160), 118.35 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (53/53), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


In [4]:
os.chdir('Tutorial_IDP_SAPs/Simulation code')

In [35]:
from global_vars import (stored_values, initialize_callbacks,
                        create_widget_layout)
from traj_process import process_trajectory

In [36]:
initialize_callbacks()
widget = create_widget_layout()

In [55]:
display(widget)

In [58]:
stored_values

{'Simulation type': 'Co-assembly',
 'Sequence': None,
 'Sequence 1': 'HWEHWEHWE',
 'Sequence 2': 'HHHWWWKKK',
 'Ratio': '5-5',
 'Processed_Ratio': '500-500',
 'peptide_pairs': 'HWEHWEHWE-HHHWWWKKK'}

In [64]:
#Check in src the file parsers.py, it will show all options available
ionic_strength='0.3'

In [65]:
# First check file
residues_file = 'src/residues.csv'

if not os.path.exists(residues_file):
    print(f"❌ Error: File '{residues_file}' not found!")
else:
    !python clean_pair_code.py --ratio {stored_values['Processed Ratio']} \
        --ionic-strength {ionic_strength} \
        --ph 7.4 \
        --peptide-pair {stored_values['peptide_pairs']} \
        --run-all \
        --residues-file 'src/residues.csv' \
        --steps 50000

Cannot import pdbfixer. This may affect 3SPN2 model.
usage: clean_pair_code.py [-h]
                          (--pickle-file PICKLE_FILE | --peptide-pair PEPTIDE_PAIR | --pairs-file PAIRS_FILE)
                          [--pair-index PAIR_INDEX] [--run-all] [--ratio RATIO] [--ph PH]
                          [--ionic-strength IONIC_STRENGTH] [--box-size BOX_SIZE]
                          [--z-size Z_SIZE] [--temperature TEMPERATURE] [--steps STEPS]
                          [--output-dir OUTPUT_DIR] --residues-file RESIDUES_FILE
clean_pair_code.py: error: argument --ionic-strength: invalid float value: '{ionic_strength}'


In [26]:
name = stored_values['peptide_pairs']

traj = md.load_dcd(f'{name}/310/{name}_slab.dcd', top=f'{name}/start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True)  # realign to the origin
# view = nglview.show_mdtraj(traj)

In [27]:
ens_file=f'{name}_ensemble.pkl'
ensembles = process_trajectory(f'{name}', name, ens_file)
if os.path.isfile(ens_file) and not ensembles:
    with open(ens_file, 'rb') as f:
        ensembles = pickle.load(f)

Successfully saved ensemble data to HWEHWEHWE-HHHWWWKKK_ensemble.pkl


In [29]:
with open('stored_values.pkl', 'wb') as handle:
    pickle.dump(stored_values, handle, protocol=pickle.HIGHEST_PROTOCOL)